In [3]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import os
from dotenv import load_dotenv
from urllib.parse import urlparse

def scrape_google_scholar(query, keywords=None, num_pages=0, output_csv="results.csv"):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }

    base_url = "https://scholar.google.com/scholar"
    all_results = []

    for page in range(0, num_pages, 1):

        params = {
            "q": query ,
            "start": page * 10  # 10 results per page
        }

        response = requests.get(base_url, params=params, headers=headers)
        print(f"[{page}] Status: {response.status_code}, Scraping URL: {response.url}")

        # print(response.url)

        soup = BeautifulSoup(response.text, "html.parser")

        results = soup.select(".gs_ri")

        for result in results:
            title_tag = result.select_one(".gs_rt")
            title = title_tag.get_text(strip=True) if title_tag else "No title"

            link_tag = title_tag.find("a") if title_tag else None
            link = link_tag["href"] if link_tag and link_tag.has_attr("href") else "No link"

            # snippet_tag = result.select_one(".gs_rs")
            # snippet = snippet_tag.get_text(strip=True) if snippet_tag else "No snippet"

            meta_tag = result.select_one(".gs_a")
            meta_text = meta_tag.get_text(strip=True) if meta_tag else ""
            authors = meta_text.split(" - ")[0] if " - " in meta_text else "No author info"

            # # Filter by keyword match in title or snippet (optional)
            # if keywords:
            #     text_combined = (title + snippet).lower()
            #     if not any(keyword.lower() in text_combined for keyword in keywords):
            #         continue  # Skip if no keywords matched
            
            domain = urlparse(link).netloc

            all_results.append({
                "Title": title,
                "Authors": authors,
                "Link": f'=HYPERLINK("{link}", "{domain}")',  # formatted hyperlink,
                # "Snippet": snippet,
                "Query": f'=HYPERLINK("{response.url}", "{response.url}")'
            })

        time.sleep(1)  # Avoid being blocked

    # Ensure output folder exists
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)

    if len(all_results) > 0:
        # Save to CSV
        with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=["Title", "Authors", "Link", "Query"], quoting=csv.QUOTE_MINIMAL)
            writer.writeheader()
            writer.writerows(all_results)

        abs_path = os.path.abspath(output_csv)
        print(f"\n✅ Scraped {len(all_results)} results and saved to:\n📂 {abs_path}")
    else:
        print('Failed to scrape papers')


if __name__ == "__main__":
    
    load_dotenv() 
    
    fixedKeywords = os.getenv("fixedKeywords")
    optionalKeywords = [kw.strip() for kw in os.getenv("optionalKeywords").split(',')]

    # ✅ Save to D:\paper\paper
    output_path = os.getenv("OutputCSVFilePath")

    # print(' OR '.join(keyword_list))
    main_query = f'{fixedKeywords} AND ({" OR ".join(optionalKeywords)})'

    scrape_google_scholar(query=main_query, num_pages=10, output_csv=output_path)


[0] Status: 429, Scraping URL: https://www.google.com/sorry/index?continue=https://scholar.google.com/scholar%3Fq%3D%2522Digital%2BTwin%2522%2BAND%2BIndustry%2BAND%2BEducation%2BAND%2B%2528Training%2BOR%2B%2522Virtual%2BReality%2522%2BOR%2B%2522Augmented%2BReality%2522%2BOR%2BVR%2BOR%2BAR%2529%26start%3D0&q=EgRn2hkhGJ659sIGIizuBATbpYw9lJdzq90FBQWgA70OUz2g5eH_IH_br4O2bI4o8AAYQ8_DlVbIwzICclJaAUM
[1] Status: 200, Scraping URL: https://scholar.google.com/scholar?q=%22Digital+Twin%22+AND+Industry+AND+Education+AND+%28Training+OR+%22Virtual+Reality%22+OR+%22Augmented+Reality%22+OR+VR+OR+AR%29&start=10
[2] Status: 200, Scraping URL: https://scholar.google.com/scholar?q=%22Digital+Twin%22+AND+Industry+AND+Education+AND+%28Training+OR+%22Virtual+Reality%22+OR+%22Augmented+Reality%22+OR+VR+OR+AR%29&start=20
[3] Status: 200, Scraping URL: https://scholar.google.com/scholar?q=%22Digital+Twin%22+AND+Industry+AND+Education+AND+%28Training+OR+%22Virtual+Reality%22+OR+%22Augmented+Reality%22+OR+VR+OR+